In [ ]:
try:
    from AtlasI2C import AtlasI2C
    import socket
    import sys
    import re
    import time
    import psycopg2
    from psycopg2 import pool
    import RPi.GPIO as GPIO
    import configparser
    import logging
    import threading
    from threading import Timer
    from datetime import datetime, timezone
    import csv
    from collections import deque
    import queue
    from queue import Queue
    import traceback
except ImportError as e:
    logging.error(f"ImportError: {e}")
    sys.exit(1)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('error.log'),
        logging.StreamHandler()
    ]
)

logging.basicConfig(level=
                    logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                    logging.FileHandler('debug.log'),
                    logging.StreamHandler()])


# GPIO pins setup
GPIO_PIN = 25      # CO2 pin
PUMP_PIN = 24      # Pin for controlling the pump c
PUMP_PIN_2 = 23    # Secondary pump control pin
HEATER_PIN = 4     # Pin for controlling the heater

# GPIO basic configurations
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)

GPIO.setup(GPIO_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN_2, GPIO.OUT)
GPIO.setup(HEATER_PIN, GPIO.OUT)


def connect_to_database():
    """
    Establishes a connection to a PostgreSQL database.
    Reads database configuration from 'config.ini' file.
    Returns:
        conn: Database connection object if successful, None otherwise.
    """
    config = configparser.ConfigParser()
    config.read('config.ini')  # Load database configuration

    host = config['database']['host']
    dbname = config['database']['dbname']
    user = config['database']['user']
    password = config['database']['password']
    sslmode = config['database']['sslmode']

    try:
        conn_string = f"host={host} user={user} dbname={dbname} password={password} sslmode={sslmode}"
        conn = psycopg2.connect(conn_string)
        logging.info("Database connection established")
        return conn
    except psycopg2.Error as e:
        logging.error(f"Database connection error: {e}")
        return None

def detectSensors():
    """
    Detects and identifies connected Atlas I2C sensors.
    Returns a list of detected sensor devices.
    """
    device_list = []
    dev = AtlasI2C()
    try:
        for address in dev.list_i2c_devices():
            dev.set_i2c_address(address)
            if 11 <= address <= 127:
                moduletype = "ERR"
                while moduletype == "ERR":
                    response = dev.query("I")
                    if ("HUM" in response):
                        moduletype = "HUM"
                    elif ("CO2" in response):
                        moduletype = "CO2"
                    elif ("PRS" in response):
                        moduletype = "PRS"
                    else:
                        error_message = "Error detecting moduletype of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)
                name = "UNKNOWN"
                while (name == "UNKNOWN") or (bool == False):
                    bool = False
                    response = dev.query("name,?")
                    if ("NAME," in response):
                        name = response.split(",")[1]
                        name = name[:9]
                        r = re.compile('.{3}-.{2}-.{2}')
                        if r.match(name):
                            bool = True
                    else:
                        error_message = "Error detecting name of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)

                device_list.append(AtlasI2C(address = address, moduletype = moduletype, name = name))

        for device in device_list:
            print("device.address="+str(device.address)+", device.moduletype="+str(moduletype)+", device.name="+str(device.name))

    except Exception as e:
        logging.error(f"Error detecting sensors: {e}")
        return []
    return device_list

def setProperties(device_list):
    try:
        for device in device_list:
            if (device.moduletype == "HUM"):
                device.write("O,T,1")
                time.sleep(1)
                device.write("O,Dew,1")
                time.sleep(1)
            if (device.moduletype == "CO2"):
                device.write("O,t,1")
                time.sleep(1)
    except Exception as e:
        logging.error(f"Error setting properties: {e}")

def writeHeader(output_file, device_list):
    try:
        csv_header = "timestamp, hostname, sensor ID, reading Type, Value"
        output_file.write(csv_header+"\n")
    except Exception as e:
        logging.error(f"Error writing header: {e}")

def requestReading(device_list):
    for device in device_list:
        try:
            device.write("R")
            time.sleep(1)
        except:
            error_message = "Error sending request to sensor "+str(device.name)+"\n"
            sys.stderr.write(error_message)
            continue

def pollDevice(device):
    reading = ""
    errMsg = ""
    try:
        reading = ""
        errMsg = ""
        try:
            device_output = device.read()
            device_output.encode('ascii')
            device_output = re.sub(r"[^a-zA-Z0-9: ,.]", "", device_output)
        except:
            print("Reading I2C device failed: "+device.name)
            return reading, str(device_output)

        if (device.moduletype == "HUM"):
            try:
                data = device_output.split(': ')[1]
                hum = float(data.split(',')[0])
                temp = float(data.split(',')[1])
                dew = float(data.split(',')[3])
            except:
                hum = 0.0
                temp = 0.0
                dew = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+ ". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device:"
            reading += str(hum) + ", "
            reading += str(temp) + ", "
            reading += str(dew)
        if (device.moduletype == "CO2"):
            try:
                data = device_output.split(': ')[1]
                co2 = float(data.split(',')[0])
                temp = float(data.split(',')[1])
            except:
                co2 = 0.0
                temp = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(co2) + ", "
            reading += str(temp)
        if (device.moduletype == "PRS"):
            try:
                data = device_output.split(': ')[1]
                prs = data
            except:
                prs = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(prs)

        return reading, str(device_output), errMsg
    except Exception as e:
        logging.error(f"Unexpected error in device poll: {e}")
        return "", "", "Unexpected error"

# CONSTANTS
N = 2
HUM_THRESHOLD_MAX = 25 # % RH
HUM_THRESHOLD_MIN = 10 # % RH
TEMP_THRESHOLD = 25 # deg C
CO2_THRESHOLD = 420 # (300 + 115)

co2_readings = deque(maxlen=N)
hum_readings = deque(maxlen=N)
temp_readingss = deque(maxlen=N)
temp_readings = deque(maxlen=N)

PUMP_ON_DURATION = 30  # Pump on for 30 sec
PUMP_ON_DURATION_2 = 30
HEATER_ON_DURATION = 1000
WARM_UP_TIME = 30  # Warm up for 3 minutes
POLL_INTERVAL = 10  # Poll every 10 seconds

CO2_CONTROL_INTERVAL = 60  # Control CO2 every 60 seconds
HUM_CONTROL_INTERVAL = 60
TEMP_CONTROL_INTERVAL = 60

co2_queue = queue.Queue()
hum_queue = queue.Queue()
temp_queue = queue.Queue()

co2_release_count = 0
dehum_release_count = 0
hum_release_count = 0
temp_release_count = 0
loop_counter = 0


# ~~~~
def turn_pump_off_after_duration(duration):
    timer = Timer(PUMP_ON_DURATION, turn_pump_off)
    timer.start()

# ~~~
def open_solenoid_valve():
    try:
        GPIO.output(GPIO_PIN, GPIO.HIGH)
        time.sleep(.06)
        GPIO.output(GPIO_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error opening solenoid valve: {e}")
        sys.exit(1)

def turn_on_dehumidifier():
    try:
        GPIO.output(PUMP_PIN, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION)
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump on: {e}")
        sys.exit(1)

def turn_on_humidifier():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION_2)
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 on: {e}")
        sys.exit(1)

def turn_off_dehumidifier():
    try:
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump off: {e}")

def turn_off_humidifier():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 off: {e}")

def turn_on_heater():
    try:
        GPIO.output(HEATER_PIN, GPIO.HIGH)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

def turn_off_heater():
    try:
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)
#~~~~~~~~

def co2_control_action():
    print("Opening CO2 valve")
    open_solenoid_valve()
    logging.info("CO2 control action triggered.")

def dehum_control_action():
    print("Turning on dehumidifier")
    turn_on_dehumidifier()
    time.sleep(PUMP_ON_DURATION)
    print("Turning off dehumidifier")

def hum_control_action():
    print("Turning on humidifier")
    turn_on_humidifier()
    time.sleep(PUMP_ON_DURATION)
    print("Turning off humidifier")

def temp_control_action():
    print("Turning on heater")
    turn_on_heater()
    time.sleep(HEATER_ON_DURATION)
    turn_off_heater()
    print("Turning off heater")



def main():

    try:
        loop_counter = 0

        conn = connect_to_database()
        if conn is None:
            sys.exit(1)
        cursor = conn.cursor()
        device_list = detectSensors()
        if (not device_list):
            sys.stderr.write("No sensors detected. Quitting...\n")
            sys.exit(1)

        setProperties(device_list)

        output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv"
        header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, \
            error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
        with open(output_filename, "w", buffering=1) as output_file:
            writeHeader(output_file, device_list)
            while True:
                temp_readings = []
                hum_readings = []
                co2_readings = []

                requestReading(device_list)
                time.sleep(1)
                for device in device_list:
                    currentReading, probeString, errorMessage = pollDevice(device)
                    data = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f ")+time.strftime("%Z")
                    date_time = datetime.now()
                    data += ", "+str(socket.gethostname()) + ", " + device.name
                    if (device.moduletype == "HUM"):
                        data1 = data + ", humidity, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", temperature," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")
                        data3 = data + ", dew point," + currentReading.split(',')[2]
                        output_file.write(str(data3)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "humidity", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "temperature", currentReading.split(',')[1], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "dew point", currentReading.split(',')[2], \
                            errorMessage, probeString))

                        hum_readings.append(float(currentReading.split(',')[0]))
                        temp_readings.append(float(currentReading.split(',')[1]))

                        #print(f"HUM level for {device.name}: {hum_readings}")
                        #print(f"TEMP level for {device.name}: {temp_readings}")

                    if (device.moduletype == "CO2"):
                        data1 = data + ", CO2, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", sensor temp," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "CO2", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "sensor temp", currentReading.split(',')[1], \
                            errorMessage, probeString))

                        co2_readings.append(float(currentReading.split(',')[0]))
                        print(f"CO2 level for {device.name}: {co2_readings}")

                    if (device.moduletype == "PRS"):
                        data1 = data + ", pressure, " + currentReading
                        output_file.write(str(data1)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "pressure", currentReading, \
                            errorMessage, probeString))

                if hum_readings:
                    avg_humidity = sum(hum_readings) / len(hum_readings)
                    hum_queue.put(("Humidity", avg_humidity))
                    print(f"HUM average  Level: {avg_humidity}")

                if temp_readings:
                    avg_temperature = sum(temp_readings) / len(temp_readings)
                    temp_queue.put(("Temperature", avg_temperature))
                    print(f"TEMP average  Level: {avg_temperature}")

                if co2_readings:
                    avg_co2 = sum(co2_readings) / len(co2_readings)
                    co2_queue.put(("CO2", avg_co2))
                    #print(f"CO2 average  Level: {avg_co2}")

                    conn.commit()

                loop_counter += 1
                print(loop_counter)
                time.sleep(10)


    except Exception as e:
        logging.error(f"Unexpected error in main loop: {e}")
    finally:
        if conn:
            conn.close()
        GPIO.cleanup()

def write_data_to_file(filename, data):
    with open(filename, 'w') as file:
        for key, value in data.items():
            file.write(f"{key}: {value}\n")

#~~~~~~~~~
def control_action(data_queue, data_type, threshold, control_action, interval, comparison_type, release_count):
    while True:
        try:
            time.sleep(interval)

            # Initialize latest_value outside of the loop
            latest_value = None

            while not data_queue.empty():
                _, latest_value = data_queue.get()  # Retrieves the most recent value from the queue

            if latest_value is not None:
                print(f"Current average {data_type} Level: {latest_value}")

                # Trigger control action based on current value and threshold
                if (comparison_type == "lower" and latest_value < threshold) or \
                   (comparison_type == "upper" and latest_value > threshold):
                    print(f"Activating {data_type} control action")
                    control_action()

                    # Update the counter
                    release_count += 1
                    print(f"{data_type} Release Count: {release_count}")

        except Exception as e:
            logging.error(f"Error in {data_type} control loop: {e}")


# Initialize a dictionary to hold data
data_to_store = {
    "Loop Counter": loop_counter,
    "CO2 Release Count": co2_release_count,
    "Dehumidifier Release Count": dehum_release_count,
    "Humidifier Release Count": hum_release_count,
    "Heater Release Count": temp_release_count,
}

output_filename = "counters.txt"

write_data_to_file(output_filename, data_to_store)


if __name__ == '__main__':
    data_queue = queue.Queue()
    device_list = detectSensors()

    # Create and start the threads
    main_thread = threading.Thread(target=main)
    co2_thread = threading.Thread(target=control_action, args=(co2_queue, "CO2", CO2_THRESHOLD, co2_control_action, CO2_CONTROL_INTERVAL, "lower", co2_release_count))
    hum_thread = threading.Thread(target=control_action, args=(hum_queue, "Humidity", HUM_THRESHOLD_MIN, hum_control_action, HUM_CONTROL_INTERVAL, "lower", hum_release_count))
    #dehum_thread = threading.Thread(target=control_action, args=(hum_queue, "Humidity", HUM_THRESHOLD_MAX, dehum_control_action, HUM_CONTROL_INTERVAL, "upper", dehum_release_count))
    temp_thread = threading.Thread(target=control_action, args=(temp_queue, "Temperature", TEMP_THRESHOLD, temp_control_action, TEMP_CONTROL_INTERVAL, "lower", temp_release_count))

    main_thread.start()
    co2_thread.start()
    #dehum_thread.start()
    hum_thread.start()
    temp_thread.start()

    main_thread.join()
    co2_thread.join()
    #dehum_thread.join()
    hum_thread.join()
    temp_thread.join()